In [31]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
from pymongo import MongoClient
import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [41]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument("lang=ko_KR")

In [30]:
client = MongoClient()

In [32]:
db = client.test

In [33]:
results = db.results

In [42]:
# 크롬 드라이버의 경로
# headless
add = '/home/jin/다운로드/chromedriver'
driver = webdriver.Chrome(add, chrome_options=options)

/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [53]:
keywords = '태풍' # 키워드
words = ['부산', '서울', '광주', '대구', '세종', '제주', '울산', '인천'] # 지역명
begin = '2016.03.11' # 시작 날 yyyy.mm.dd
end = '2018.06.15' # 끝 날 yyyy.mm.dd

In [54]:
url = 'https://search.naver.com/search.naver?where=news&query='+ keywords \
    +'&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=' \
    + begin + '&de=' + end + \
    '&docid=&nso=&mynews=0&refresh_start=0&related=0'

In [55]:
driver.get(url)

In [28]:
linkList = driver.find_elements_by_class_name("_sp_each_url")
sources = driver.find_elements_by_class_name("_sp_each_source")

for i, link in enumerate(linkList):
    if link.text == '네이버뉴스':
        print(i)

7


In [25]:
lst = driver.find_elements_by_tag_name('a')
for i in lst:
    if i.text == '네이버뉴스':
        print(i.get_attribute('href'))

https://search.naver.com/p/crd/rd?m=1&px=0&py=0&sx=0&sy=0&p=U7a+AwpVuEVssbdid28ssssstos-344550&q=%C8%AB%BC%F6&ssc=tab.news.all&f=news&w=news&s=WXC6/B2ukq2ukvh+Wllh7g==&time=1555136143209&a=nws*a.nav&r=26&i=88000108_000000000000000003319592&u=https%3A//news.naver.com/main/read.nhn%3Fmode%3DLSD%26mid%3Dsec%26sid1%3D104%26oid%3D011%26aid%3D0003319592
https://search.naver.com/p/crd/rd?m=1&px=0&py=0&sx=0&sy=0&p=U7a+AwpVuEVssbdid28ssssstos-344550&q=%C8%AB%BC%F6&ssc=tab.news.all&f=news&w=news&s=WXC6/B2ukq2ukvh+Wllh7g==&time=1555136143260&a=nws*c.nav&r=27&i=880000D8_000000000000000010142682&u=https%3A//news.naver.com/main/read.nhn%3Fmode%3DLSD%26mid%3Dsec%26sid1%3D104%26oid%3D001%26aid%3D0010142682
https://search.naver.com/p/crd/rd?m=1&px=0&py=0&sx=0&sy=0&p=U7a+AwpVuEVssbdid28ssssstos-344550&q=%C8%AB%BC%F6&ssc=tab.news.all&f=news&w=news&s=WXC6/B2ukq2ukvh+Wllh7g==&time=1555136143302&a=nws*e.nav&r=28&i=880000BC_000000000000000004166430&u=https%3A//news.naver.com/main/read.nhn%3Fmode%3DLSD%26mid%

In [39]:
for t in linkList:
    print(t.text)

네이버뉴스
네이버뉴스
하수처리수 공업용수 재이용사업 본격화
'수해 안전 경기도'‥선진적 하천관리방안 모색
네이버뉴스
네이버뉴스
완도군, 수산물 소비 촉진 종합대책 간담회 개최
팜한농, 태광배추·탐스런토마토 '주목'
예산군 용굴천 지방하천 정비사업 순항
경기도, 자연재난 안전지킴이 시범사업 추진....지역 주민 활용키로
[발언대] 오늘 16세 소녀를 가슴 뛰게 하는 것


<h3>4000개 페이지 까지만 크롤링 가능. 날자 이용해서 이어나가야할듯<br>
수정해야할거 개많당..
</h3>

In [56]:
for i in range(1000):
    linkList = driver.find_elements_by_class_name("_sp_each_url")
    sources = driver.find_elements_by_class_name("_sp_each_source")

    for i, link in enumerate(linkList):
        if link.text == '네이버뉴스':
#             source = sources[i].text # 출처
            link.send_keys(Keys.ENTER)
            try:
                driver.switch_to_window(driver.window_handles[1])

                element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, "articleTitle"))) # 로딩이 안되면 5초 기다려줍니다

                title = driver.find_element_by_id("articleTitle").text # 제목
                content = driver.find_element_by_id("articleBodyContents").text # 본문
                date = driver.find_element_by_class_name("t11").text # 게시일
                url = driver.current_url # 현재 URL

                for term in words: 
                    if term not in content:
                        words.remove(term)
                
                result = {
                    'title' : title,
                    'url' : url,
                    'date' : date,
                    'keywords' : keywords,
                    'beloningWords' : words,
                    'begin-date' : begin,
                    'end-date' : end,
                    'contents' : content
                }
                
                results.insert(result)
                
                driver.close()
                driver.switch_to_window(driver.window_handles[0])
                
            except TimeoutException as e:
                driver.close()
                driver.switch_to_window(driver.window_handles[0])
            
            finally:
                
                sleep(1)
                
    # 다음 페이지로 이동            
    driver.find_element_by_class_name('next').send_keys(Keys.ENTER)
    sleep(1)

# driver.quit()

/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use driver.switch_to.window instead
  # Remove the CWD from sys.path while we load stuff.
/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: use driver.switch_to.window instead
/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: use driver.switch_to.window instead


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","selector":"next"}
  (Session info: headless chrome=72.0.3626.121)
  (Driver info: chromedriver=2.42.591071 (0b695ff80972cc1a65a5cd643186d2ae582cd4ac),platform=Linux 4.15.0-47-generic x86_64)


In [173]:
driver.switch_to_window(driver.window_handles[0])

/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


<h2>크롤러 고도화를 위한 코드(ongoing)</h2>


In [131]:
for link in linkList[1:]:
    
    link.send_keys(Keys.ENTER) # 해당 뉴스 클릭
    try:
        elem = WebDriverWait(driver, 7).until(EC.presence_of_element_located())
    except TimeoutException:
        
    driver.switch_to_window(driver.window_handles[1]) # 드라이버 시점 전환
    
    try:
        txt = driver.find_element_by_xpath("//div[contains(., 'article')]").text
    except:
        txt = driver.find_element_by_xpath("//div[contains(., 'content')]").text
    
    contents = list(filter(None, txt.split('\n')))
    
    filtered = []
    for content in contents:
        beginFlag, endFlag = False, False
        queue, cnt = [], 0
        
        if pattern.match(content) != None:
            beginFlag = True
        if len(content) > 50:
            endFlag = True
            
        if beginFlag == True and len(content) > 10:
            filtered.append(content)
                    
        # 기사의 끝을 알기 위한 조건
        if beginFlag == True and '@' in content:
            break
        # 기자의 메일이 없는 경우
        if endFlag == True and len(content) < 10:
            queue.append(content)
            cnt = cnt + 1
                
        if cnt == 3:
            break
    print(filtered)
                    
    driver.switch_to_window(driver.window_handles[0]) # 뉴스 리스트 창으로 화면 전환
    

/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use driver.switch_to.window instead
  """


[]


/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: use driver.switch_to.window instead


[]
[]
[]
[]
[]
[]
[]
[]
